# Рынок заведений общественного питания Москвы

### Цель исследования:

Подготовить исследование рынка Москвы, найти интересные особенности и презентовать полученные результаты, которые в будущем помогут в выборе подходящего инвесторам места.
Помочь определиться что это будет за место: кафе, ресторан, пиццерия, паб или бар, — и какими будут расположение, меню и цены.

### Располагаемые данные:

Датасет с заведениями общественного питания Москвы: файл moscow_places.csv
* name — название заведения;
- address — адрес заведения;
- category — категория заведения, например «кафе», «пиццерия» или «кофейня»;
- hours — информация о днях и часах работы;
- lat — широта географической точки, в которой находится заведение;
- lng — долгота географической точки, в которой находится заведение;
- rating — рейтинг заведения по оценкам пользователей в Яндекс Картах (высшая оценка — 5.0);
- price — категория цен в заведении, например «средние», «ниже среднего», «выше среднего» и так далее;
- avg_bill — строка, которая хранит среднюю стоимость заказа в виде диапазона, например:

«Средний счёт: 1000–1500 ₽»;

«Цена чашки капучино: 130–220 ₽»;

«Цена бокала пива: 400–600 ₽».
и так далее;

- middle_avg_bill — число с оценкой среднего чека, которое указано только для значений из столбца avg_bill, начинающихся с подстроки «Средний счёт»:

Если в строке указан ценовой диапазон из двух значений, в столбец войдёт медиана этих двух значений.

Если в строке указано одно число — цена без диапазона, то в столбец войдёт это число.

Если значения нет или оно не начинается с подстроки «Средний счёт», то в столбец ничего не войдёт.

- middle_coffee_cup — число с оценкой одной чашки капучино, которое указано только для значений из столбца avg_bill, начинающихся с подстроки «Цена одной чашки капучино»:

Если в строке указан ценовой диапазон из двух значений, в столбец войдёт медиана этих двух значений.

Если в строке указано одно число — цена без диапазона, то в столбец войдёт это число.

Если значения нет или оно не начинается с подстроки «Цена одной чашки капучино», то в столбец ничего не войдёт.
- chain — число, выраженное 0 или 1, которое показывает, является ли заведение сетевым (для маленьких сетей могут встречаться ошибки):

0 — заведение не является сетевым

1 — заведение является сетевым

- district — административный район, в котором находится заведение, например Центральный административный округ;
- seats — количество посадочных мест.

### Ход исследования: 

- Обзор данных
- Предобработка данных
- Анализ данных
- Детализация исследование: открытие кофейни
- Подготовка к презентации
- Выводы

### Откроем файл с данными и изучим общую информацию

In [1]:
# импорт библиотек
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats as st
import seaborn as sns
from datetime import datetime, timedelta
from plotly import graph_objects as go
import plotly.express as px
import folium
import json
from folium import  Choropleth
from folium.plugins import MarkerCluster
from folium import Map, Marker

ModuleNotFoundError: No module named 'folium'

In [ ]:
df = pd.read_csv('/datasets/moscow_places.csv')
display(df.head(10))
display(df.info())

In [ ]:
df.describe()

#### Вывод : 
Открыв файл с данными мы можем заметить пропуски в некоторых столбцах на первый взгляд бросаются категория цены, средняя цена за бокал пива, средняя цена за чашку кофе,средний чек и кол-во посадочных мест. В дальнейшем мы проверим сможем ли мы что нибудь с этим сделать. С типами данных все в порядке. Приведем название заведений и адрес к нижнему регистру для удобства предобработки, проверим данные на явные и не явные дубликаты. На данный момент можем заметить что у нас 8406 заведений

### Предобработка данных

In [ ]:
# Приведем названия заведений и адреса к нижнему регистру
df.name = df['name'].str.lower()
df.address = df['address'].str.lower()
df.head()

In [ ]:
#Выполним проверку на явные дубликаты
df.duplicated().sum()

Проверим нет ли дубликатов заведений по одному и тому же адресу

In [ ]:
#Выполним проверку на неявные дубликаты
df.duplicated(subset=['name', 'address']).sum()

Видим что есть пересечения. Удалим эти дубликаты

In [ ]:
#Удалим неявные дубликаты
df = df.drop_duplicates(subset=['name', 'address'])
df.duplicated(subset=['name', 'address']).sum()

In [ ]:
# Проверим данные на пропуски: 

df.isnull().sum()

In [ ]:
# Проверим долю пропусков от общего числа 
round(df.isnull().sum()/len(df) * 100,2)

Можем заметить что пропуски занимают крайне большую доню в данных, удалить их просто так не получится. Заменить на среднее или медианное значение тоже сомнительная идея. Например заведения могут слишком отличаться и форматом и кол-вом посадочных мест, наверное в данном случае пропуски стоит оставить так как есть.

#### Создадим столбец street с названиями улиц из столбца с адресом.

In [ ]:
#разделим подстроку с помощью метода str.split и возьмем 2-й элемент
df['street'] = df['address'].str.split(', ').str[1]
df.head()

###### Создадим столбец is_24/7 с обозначением, что заведение работает ежедневно и круглосуточно (24/7):
* логическое значение True — если заведение работает ежедневно и круглосуточно;
* логическое значение False — в противоположном случае.

In [ ]:
#Воспользуемся методом str.contains, если он увидит нужные нам строки, значит значение в новом столбце будет True
df['is_24/7'] = df['hours'].str.contains('ежедневно, круглосуточно')
df.head()

#### Вывод: 
Провели предобработку данных, привели к нижнему регистру названия заведений и адресс для удобства. Выяснили сколько пропуски занимают в процентном соотношении от общего кол-ва и поняли что не сможем их заменить либо удалить, провели проверку на явные и неявные дубликаты и избавились от них. Добавили столбцы с улицами заведений и с заведениями, которые работают круглосуточно и ежедневно, если заведение работает в таком режиме в столбце оно будет указано как True, если нет - False

### Анализ данных

#### Проверим какие категории заведений представлены в данных? Исследуем количество объектов общественного питания по категориям: рестораны, кофейни, пиццерии, бары и так далее. Построим визуализации. Ответим на вопрос о распределении заведений по категориям.

In [ ]:
#Создадим новую переменную в которой сгрупируем категории и отсортируем от большего кол-ва к меньшему
df_cat = df.groupby('category')['name'].count().reset_index().sort_values(by='name',ascending=False)
df_cat

По табличке можем увидеть, что больше всего заведений под категорией "кафе" и "ресторан". Меньше всего столовых и булочных
Построим визуализацию 

In [ ]:
#строим столбчатую диаграмму
fig = px.bar(df_cat, x='category', y= 'name', title = 'Кол-во заведений по категориям',color_discrete_sequence = px.colors.sequential.Rainbow)
fig.update_layout(xaxis_title = 'Категории заведений',
                  yaxis_title = 'Кол-во заведений',
                  legend_title_text = 'Категории')


fig.show()

Построили столбчатую диаграмму, выделили лидеров и аутсайдеров.
Кафе и рестораны в лидерах, по моим предположениям это является одни из самых популярных проведения досуга и пользуется большим спросом, на 3 месте находятся кофейни тоже с не плохим кол-вом, люди пьют много кофе, по пути на работу, в перерывах или по пути на встречу. Столовые и булочные отстают, по моим предположениям столовые скорее находятся где-то рядом с предприятиями, возможно производствами, или учебными заведениями, по своему опыту могу сказать, что в центре их практически не найти

#### Исследуем количество посадочных мест в местах по категориям: рестораны, кофейни, пиццерии, бары и так далее. Построим визуализации. Проанализируем результаты и сделаем выводы.

In [ ]:
#Создадим новую переменную в которой сгрупируем категории и отсортируем от большего кол-ва посадочных мест к меньшему
df_seats = df.groupby('category')['seats'].median().reset_index().sort_values(by='seats',ascending=False)
df_seats

Можем заметить что рестораны,бары\пабы и кофейни лидируют по кол-ву посадочных мест. Меньше всего в пиццериях и булочных.
Построим визуализацию


In [ ]:
#строим столбчатую диаграмму
fig = px.bar(df_seats, x='category', y= 'seats', title = 'Кол-во посадочных мест по категориям', \
             color='category')
fig.update_layout(xaxis_title = 'Посадочные места',
                  yaxis_title = 'Категории',
                  legend_title_text = 'Категории')

fig.show()

У лидеров(рестораны и бары\пабы) ситуация по рассадке практическа равная, так же как и аутсайдеров (булочная и пиццерия).
Что в целом логично, заведения по типу первых двух проводят банкеты,дни рождения,свадьбы,в пятницу люди часто могут проводить там время после работы что бы расслабиться и отвлечься там часто требуется большая вместимость, в отличии от той же булочной,зачем там много посадочных мест? Этот тип заведения в основном работает на вынос.

#### Рассмотрим и изобразим соотношение сетевых и несетевых заведений в датасете. Проверим каких заведений больше?

In [ ]:
#выведем кол-во сетевых и не сетевых заведений
df_chain = df['chain'].value_counts().reset_index()
df_chain = df_chain.rename(columns={'index': 'chain', 'chain': 'count'})
df_chain['chain'] = df_chain['chain'].map({ 0:'несетевое', 1:'сетевое'})
df_chain

По табличке видим что несетевых заведений больше
Построим визуализацую

In [ ]:
#Строим круговую диаграмму
fig = go.Figure(data=[go.Pie(labels=df_chain['chain'], values=df_chain['count'],\
                             title ='Соотношение сетевых и несетевых заведений',
                             
                             pull=[0.1,0])])
fig.show()

Можем увидеть что несетевые составляют 61.9%, сетейвые 38.1%

#### Выясним какие категории заведений чаще являются сетевыми? Исследуем данные и ответим на вопрос графиком.

In [ ]:
#Создадим новый столбец с сетевыми и несетевыми завеедниями
df['chain_new'] = df['chain'].map({ 0:'несетевое', 1:'сетевое'})
df.head()

In [ ]:
#Построим сводную таблицу и посмотрим сколько сетевых и не сетевых заведений по категориям
df_chain_1 = df.groupby(['category','chain_new']).agg({'chain': 'count'}).reset_index()
df_chain_1

Что ж, глядя на таблицу можем заметить, что в некоторых категориях преобладают сети, построим график и посмотрим как это выглядит

In [ ]:
df_chain_new = df.groupby(['category', 'chain_new'])
df_chain_new_1 = pd.DataFrame({'count': df_chain_new.size()})
df_chain_new_1['perc'] = (df_chain_new_1['count'] / df_chain_new_1.groupby(level=0)['count'].sum() * 100).round(2)
df_chain_new_1 = df_chain_new_1.reset_index()
df_chain_new_2 = df_chain_new_1[df_chain_new_1['chain_new'] == 'сетевое']
df_chain_new_1

In [ ]:
#Строим столбчатую диаграмму
fig = px.bar(df_chain_new_1, x='category', y= 'perc', title = 'Соотношение сетевых и не сетевых заведений', \
             color = 'chain_new')
fig.update_layout(xaxis_title = 'Категории заведений',
                  yaxis_title = 'Доля заведений',
                  legend_title_text = 'Соотношение',
                  yaxis = {"categoryorder":"total descending"})
fig.update_traces(hovertemplate = 'Категория заведения : %{x} <br>Доля: %{y}')

fig.show()

In [ ]:
#Строим столбчатую диаграмму
fig = px.bar(df_chain_new_2, x='category', y= 'perc', title = 'Доля сетевых заведений', \
             color_discrete_sequence = px.colors.sequential.RdBu)
fig.update_layout(xaxis_title = 'Категории заведений',
                  yaxis_title = 'Доля заведений',
                  legend_title_text = 'Соотношение',
                  xaxis = {"categoryorder":"total descending"})
fig.update_traces(hovertemplate = 'Категория заведения : %{x} <br>Доля: %{y}')

fig.show()


На диаграмме можем заметить что кафе и рестораны в большинстве своем не являются сетью.
В кофейнях,пиццериях и булочных как раз упор идет больше на сеть

#### Сгруппируем данные по названиям заведений и найдем топ-15 популярных сетей в Москве. Под популярностью понимается количество заведений этой сети в регионе. Построим подходящую для такой информации визуализацию. Знакомы ли нам эти сети? Есть ли какой-то признак, который их объединяет? К какой категории заведений они относятся?

In [ ]:
df_1 = df[df['chain'] == 1]

top_16 = df_1.groupby('name')['address'].count().reset_index().sort_values(by='address',ascending=False).head(15)

top_17 = df_1[df_1['name'].isin(top_16['name'])] 

top_15_new = top_17.groupby(['name','category']).agg({'address':'count'}).reset_index().sort_values(by='address',ascending=False)
top_15_new = top_15_new.rename(columns={'address': 'count'})
top_15_new

In [ ]:
#строим столбчатую диаграмму
fig = px.bar(top_15_new, x='count', y='name', color='category', title='Топ 15 сетей')
fig.update_xaxes(tickangle=45)
fig.update_layout(xaxis_title = 'Заведения',
                  yaxis_title = 'Количество заведений',
                  legend_title_text = 'Категории',
                  yaxis = {"categoryorder":"total ascending"})

fig.show() 

Глядя на таблицу сразу можно сказать что это очень известные сети возможно даже не Москвы,а всей РФ в целом. Что сразу бросается в глаза, так это то что самые крупные сети в основном являются кофейнями либо же пиццериями.
Как ранее мы видели именно эти категории преобладают сетями

На графике можем заметить что большего всего представителей в топ 15 - это кофейни, по 3 представителя у кафе и ресторанов, не смотря на то что что у пиццерии 2 представителя, кол-во их заведений превосходит многие заведения. Так же в наш рейтинг попала популярная булочная "буханка"

#### Посмотрим какие административные районы Москвы присутствуют в датасете? Отобразим общее количество заведений и количество заведений каждой категории по районам. Попробуем проиллюстрировать эту информацию одним графиком.

In [ ]:
#Отобразим общее количество заведений и количество заведений каждой категории по районам
df_dis = df.groupby(['district','category'])['name'].count().reset_index()
df_dis = df_dis.sort_values(by=['district','name'],ascending=False)
df_dis

Глядя на таблицу пока не понятно где, что преобладает, поэтому построим график.

In [ ]:
#строим столбчатую диаграмму
fig = px.bar(df_dis, x='name', y= 'district', title = 'Количество заведений в административных округах Москвы по категориям', \
             color='category')
fig.update_layout(xaxis_title = 'Административнй округ',
                  yaxis_title = 'Количество заведений',
                 legend_title_text = 'Категории')
fig.update_layout(yaxis = {"categoryorder":"total ascending"})

fig.show()

Глядя на график можем сразу сказать, что практически во всех округах доминируют кафе и рестораны,во всех округах кроме центрального лидируют кафе, в центральном округе лидируют рестораны с солидным отрывом.Так же можем заметить что в центральном округе больше всего баров и пабов, что логично, люди всегда едут гулять и отдыхать в центр. Так же значительно больше в центре кофеен,возможно связано с большим кол-вом офисных зданий.

#### Визуализируем распределение средних рейтингов по категориям заведений.  Проверим сильно ли различаются усреднённые рейтинги в разных типах общепита?

In [ ]:
#Cгрупируем данные по категориям и проверим средний рейтинг заведений
df_rating = df.groupby('category')['rating'].mean().reset_index().sort_values(by='rating',ascending=False).round(2)
df_rating

Видим что лучший рейтинг в среднем получают развлекательные заведения, а именно бары и пабы.
Самый низкий рейтинг в среднем получают заведения быстрого питания,возможно у них упор идет больше не на качество,а на кол-во посетителей

In [ ]:
#Построим столбчатую диаграмму 
fig = px.bar(df_rating, x='category', y= 'rating', title = 'Распределение средних рейтингов по категориям заведений',\
             color='category')

fig.update_layout(xaxis_title = 'Категории',
                  yaxis_title = 'Рейтинг',
                  legend_title_text = 'Категории')
fig.update_yaxes(range=[3.5, 4.5])

fig.show()

Глядя на график можем сказать что у всех категорий заведений в среднем довольно не плохой рейтинг, нет ни одной категориий с средним рейтингом ниже 4. Самый высокий рейтинг у баров и пабов - 4.38. Можем сделать вывод что усредненные рейтинги отличаются не сильно

#### Построим фоновую картограмму (хороплет) со средним рейтингом заведений каждого района. Границы районов Москвы, которые встречаются в датасете, хранятся в файле admin_level_geomap.geojson

In [ ]:
#Сгрупируем средний рейтинг заведений по районам
df_rating_district = df.groupby('district')['rating'].mean().reset_index().sort_values(by='rating',ascending=False).round(2)
df_rating_district

Можем заметить что в центральном округе в среднем оценки выше

Построим фоновую картограмму (хороплет)

In [ ]:
# загружаем JSON-файл с границами округов Москвы
state_geo = '/datasets/admin_level_geomap.geojson'

# moscow_lat - широта центра Москвы, moscow_lng - долгота центра Москвы
moscow_lat, moscow_lng = 55.751244, 37.618423

# создаём карту Москвы
m = Map(location=[moscow_lat, moscow_lng], zoom_start=10, tiles='Cartodb Positron')

# создаём хороплет с помощью конструктора Choropleth и добавляем его на карту
Choropleth(
    geo_data=state_geo,
    data=df_rating_district,
    columns=['district', 'rating'],
    key_on='feature.name',
    fill_color='OrRd',
    fill_opacity=0.8,
    legend_name='Медианный рейтинг заведений по районам',
).add_to(m)

# выводим карту
m

Построили фоновую картограмму(хороплет) и можем на ней заметить, в каких округах средние оценки выше

#### Отобразим заведения датасета на карте с помощью кластеров 

In [ ]:
# moscow_lat - широта центра Москвы, moscow_lng - долгота центра Москвы
moscow_lat, moscow_lng = 55.751244, 37.618423

# создаём карту Москвы
m = Map(location=[moscow_lat, moscow_lng], zoom_start=10, tiles="Cartodb Positron")
# создаём пустой кластер, добавляем его на карту
marker_cluster = MarkerCluster().add_to(m)

# пишем функцию, которая принимает строку датафрейма,
# создаёт маркер в текущей точке и добавляет его в кластер marker_cluster
def create_clusters(row):
    Marker(
        [row['lat'], row['lng']],
        popup=f"{row['name']} {row['rating']}",
    ).add_to(marker_cluster)

# применяем функцию create_clusters() к каждой строке датафрейма
df.apply(create_clusters, axis=1)

# выводим карту
m

По кластерам хорошо видно, что заведения в центре преобладают

#### Найдем топ-15 улиц по количеству заведений. Построим график распределения количества заведений и их категорий по этим улицам. Попробуйте проиллюстрировать эту информацию одним графиком.

In [ ]:
#Найдем топ-15 улиц по количеству заведений
df_street = df.groupby('street')['name'].count().reset_index().sort_values(by='name',ascending=False).head(15)
df_street

По табличке видно что на проспекте мира больше всего заведений, что не удивительно, это одно из самых популярных и проходных мест в Москве. 

Теперь из этих улиц нам нужно разбить заведения по категориям

In [ ]:
#Сгруппируем улицы по количеству заведений с их категориями 
df_street_1 = df.groupby(['street','category'])['name'].count().reset_index().sort_values(by='name',ascending=False)

#Заменим все улицы на улицы из топ 15
df_street_2 = df_street_1[df_street_1['street'].isin(df_street['street'])] 
df_street_2 = df_street_2.sort_values(by='name',ascending=False)
df_street_2

Проилюстрируем на графике

In [ ]:
#строим столбчатую диаграмму
fig = px.bar(df_street_2, x='name', y= 'street', title = 'топ-15 улиц по количеству заведений и по категориям', \
             color='category')
fig.update_layout(xaxis_title = 'Кол-во заведений',
                  yaxis_title = 'Название улиц',
                  legend_title_text = 'Категории')
fig.update_layout(yaxis = {"categoryorder":"total ascending"})

fig.show()

На графике наглядно видно,каких заведений на каких улицах больше, например на самой популярной (проспект мира) преобладают кафе,а на ленинградском проспекте преобладают рестораны и кофейни.

#### Найдем улицы, на которых находится только один объект общепита. 

In [ ]:
#Найдем улицы, на которых находится только один объект общепита.
df_name = df.groupby('street')['name'].count().reset_index().sort_values(by='name',ascending=False)
df_name = df_name[df_name['name'] < 2]
df_name

На табличке можем увидеть 457 результатов.

По этой таблице пока сложно сделать какие либо выводы

In [ ]:
#добавим категории заведений на этих улицах
df_name_2 = df_street_1[df_street_1['street'].isin(df_name['street'])]
df_name_2 = df_name_2.groupby('category')['name'].count().reset_index().sort_values(by='name',ascending=False)
df_name_2

In [ ]:
#строим столбчатую диаграмму
fig = px.bar(df_name_2, x='category', y= 'name', title = 'Улица,где находится только одно заведение в разрезе категорий', \
             color='category')
fig.update_layout(xaxis_title = 'Категории',
                  yaxis_title = 'Кол-во заведений')

fig.show()

В разрезе категорий можем сказать,что больше всего по 1 заведению на 1 улице находится кафе,их практически в 2 раза больше чем у ближайших преследователей. Рестораны и кофейни держатся практически на равне, остальные сильно отстали.
Из этого можно предположить,что возможно они находятся в часто проходимых местах,возможно это какие-то ЖК,возможно это какие-то заведение под особую тематику,точно не в ТЦ.


#### Посчитаем медиану среднего чека для каждого района. Используем это значение в качестве ценового индикатора района. Построим фоновую картограмму (хороплет) с полученными значениями для каждого района. Проанализируем цены в центральном административном округе и других. Проверим как удалённость от центра влияет на цены в заведениях.

In [ ]:
#Проверим медианные цены заведений по районам
df_avg = df.groupby('district')['middle_avg_bill'].median().reset_index().sort_values(by='middle_avg_bill',ascending=False)
df_avg

По табличке видим что самый высокий медианный чек у Западного административного округа и Центрального административного округа
Самый низкий у Юго-Восточного административного округа

Построим фоновую картограмму (хороплет) с полученными значениями для каждого района.


In [ ]:
# загружаем JSON-файл с границами округов Москвы
state_geo = '/datasets/admin_level_geomap.geojson'

# moscow_lat - широта центра Москвы, moscow_lng - долгота центра Москвы
moscow_lat, moscow_lng = 55.751244, 37.618423

# создаём карту Москвы
m = Map(location=[moscow_lat, moscow_lng], zoom_start=10, tiles='Cartodb Positron')

# создаём хороплет с помощью конструктора Choropleth и добавляем его на карту
Choropleth(
    geo_data=state_geo,
    data=df_avg,
    columns=['district', 'middle_avg_bill'],
    key_on='feature.name',
    fill_color='YlGn',
    fill_opacity=0.8,
    legend_name='Медианный средний чек по районам',
).add_to(m)

# выводим карту
m

Можем заметить исходя из полученных данных что медианный чек у Западного административного округа и Центрального административного округа Самый низкий у Юго-Восточного административного округа. Цены отличаются ощутимо, в некотрых районах разница в 1.5-2 раза. В центрально округе расположены самые дорогие и знаменитые завединя общественного питания, центр более туристическое место, более проходимое, так же там много офисных зданий, где неподалеку могут проходить встречи и тд. В западном регионе ситуация похожая, там находится много парков,деловых центров, элитных районов. Тот же юго-восточный район является больше спальным районом,там меньше заведений, меньше развлечений, подешевле цены на недвижимость,в общем в целом этот район можно назвать чуть проще.

# Вывод: 

- Кафе и рестораны в лидерах, по моим предположениям это является одни из самых популярных проведения досуга и пользуется большим спросом, на 3 месте находятся кофейни тоже с не плохим кол-вом, люди пьют много кофе, по пути на работу, в перерывах или по пути на встречу. Столовые и булочные отстают, по моим предположениям столовые скорее находятся где-то рядом с предприятиями, возможно производствами, или учебными заведениями, по своему опыту могу сказать, что в центре их практически не найти

- У лидеров(рестораны и бары\пабы) ситуация по рассадке практическа равная, так же как и аутсайдеров (булочная и пиццерия). Что в целом логично, заведения по типу первых двух проводят банкеты,дни рождения,свадьбы,в пятницу люди часто могут проводить там время после работы что бы расслабиться и отвлечься там часто требуется большая вместимость, в отличии от той же булочной,зачем там много посадочных мест? Этот тип заведения в основном работает на вынос.

- Мы выяснили что несетевые завединия составляют 61.9%, сетевые 38.1%
- Мы выяснили что кафе и рестораны в большинстве своем не являются сетью. В кофейнях,пиццериях и булочных как раз упор идет больше на сеть
- Мы выявили топ 15 сетевых заведений по популярности и оказалось что большего всего представителей в топ 15 - это кофейни, по 3 представителя у кафе и ресторанов, не смотря на то что что у пиццерии 2 представителя, кол-во их заведений превосходит многие заведения. Так же в наш рейтинг попала популярная булочная "буханка"
- Мы выяснили что практически во всех округах доминируют кафе и рестораны,во всех округах кроме центрального лидируют кафе, в центральном округе лидируют рестораны с солидным отрывом.Так же можем заметить что в центральном округе больше всего баров и пабов, что логично, люди всегда едут гулять и отдыхать в центр. Так же значительно больше в центре кофеен,возможно связано с большим кол-вом офисных зданий.
- Так же выявили что у всех категорий заведений в среднем довольно не плохой рейтинг, нет ни одной категориий с средним рейтингом ниже 4. Самый высокий рейтинг у баров и пабов - 4.38. Можем сделать вывод что усредненные рейтинги отличаются не сильно.
- Построили фоновую картограмму(хороплет) и увидели, в каких округах средние оценки выше
- Отобразили заведения датасета на карте с помощью кластеров
- Нашли топ-15 улиц по количеству заведений, выяснили каких заведений на каких улицах больше, например на самой популярной (проспект мира) преобладают кафе,а на ленинградском проспекте преобладают рестораны и кофейни.
- Нашли улицы на которых находится только один объект общепита. В разрезе категорий можем сказать,что больше всего по 1 заведению на 1 улице находится кафе,их практически в 2 раза больше чем у ближайших преследователей. Рестораны и кофейни держатся практически на равне, остальные сильно отстали. Из этого можно предположить,что возможно они находятся в часто проходимых местах,возможно это какие-то ЖК,возможно это какие-то заведение под особую тематику,точно не в ТЦ.
- Проанализировали цены в центральном административном округе и других. Можем заметить исходя из полученных данных что медианный чек у Западного административного округа и Центрального административного округа Самый низкий у Юго-Восточного административного округа. Цены отличаются ощутимо, в некотрых районах разница в 1.5-2 раза. В центрально округе расположены самые дорогие и знаменитые завединя общественного питания, центр более туристическое место, более проходимое, так же там много офисных зданий, где неподалеку могут проходить встречи и тд. В западном регионе ситуация похожая, там находится много парков,деловых центров, элитных районов. Тот же юго-восточный район является больше спальным районом,там меньше заведений, меньше развлечений, подешевле цены на недвижимость,в общем в целом этот район можно назвать чуть проще.

# Детализируем исследование: открытие кофейни

#### Выясним сколько всего кофеен в датасете? В каких районах их больше всего, каковы особенности их расположения?

In [ ]:
#Отсеем все категории кроме кофейни
df_coffee = df[df['category'] == 'кофейня']
#Проверим сколько всего кофеен в датасете
print('Всего кофеен в дата сете - ',len(df_coffee['category']))
#Проверим в каких районах их больше всего
df_coffee_1 = df_coffee.groupby('district')['category'].count().reset_index().sort_values(by='category',ascending=False)
df_coffee_1

Можем заметить, что с большим отрывом лидирует центральный округ по кол-ву кофеен

Построим фоновую картограмму (хороплет)



In [ ]:
# загружаем JSON-файл с границами округов Москвы
state_geo = '/datasets/admin_level_geomap.geojson'

# moscow_lat - широта центра Москвы, moscow_lng - долгота центра Москвы
moscow_lat, moscow_lng = 55.751244, 37.618423

# создаём карту Москвы
m = Map(location=[moscow_lat, moscow_lng], zoom_start=10, tiles='Cartodb Positron')

# создаём хороплет с помощью конструктора Choropleth и добавляем его на карту
Choropleth(
    geo_data=state_geo,
    data=df_coffee_1,
    columns=['district', 'category'],
    key_on='feature.name',
    fill_color='OrRd',
    fill_opacity=0.8,
    legend_name='Кол-во кофеен по районам',
).add_to(m)

# выводим карту
m

Общее кол-во кофеен по городу - 1413 из них практически треть находится в центральном округе,что свидетельствует о большой конкуренции в данном округе

#### Проверим есть ли круглосуточные кофейни?

In [ ]:
#Кол-во круглосуточных кофеен
df_coffee_24_7 = df_coffee[df_coffee['is_24/7'] == True]
print('Круглосуточных кофеен -',len(df_coffee_24_7))

Отобразим круглосуточные кофейни на карте с помощью кластеров

In [ ]:
# moscow_lat - широта центра Москвы, moscow_lng - долгота центра Москвы
moscow_lat, moscow_lng = 55.751244, 37.618423

# создаём карту Москвы
m = Map(location=[moscow_lat, moscow_lng], zoom_start=10, tiles="Cartodb Positron")
# создаём пустой кластер, добавляем его на карту
marker_cluster = MarkerCluster().add_to(m)

# пишем функцию, которая принимает строку датафрейма,
# создаёт маркер в текущей точке и добавляет его в кластер marker_cluster
def create_clusters(row):
    Marker(
        [row['lat'], row['lng']],
        popup=f"{row['name']} {row['rating']}",
    ).add_to(marker_cluster)

# применяем функцию create_clusters() к каждой строке датафрейма
df_coffee_24_7.apply(create_clusters, axis=1)

# выводим карту
m

Глядя на карту и на наши кластеры можем сказать,что больше всего круглосуточных кофеен так же находятся в центре

#### Проверим какие у кофеен рейтинги? Как они распределяются по районам?

In [ ]:
df_coffee_rat = df_coffee.groupby('district')['rating'].mean().reset_index().sort_values(by='rating',ascending=False).round(3)
df_coffee_rat

Построим фоновую картограмму (хороплет)

In [ ]:
# загружаем JSON-файл с границами округов Москвы
state_geo = '/datasets/admin_level_geomap.geojson'

# moscow_lat - широта центра Москвы, moscow_lng - долгота центра Москвы
moscow_lat, moscow_lng = 55.751244, 37.618423

# создаём карту Москвы
m = Map(location=[moscow_lat, moscow_lng], zoom_start=10, tiles='Cartodb Positron')

# создаём хороплет с помощью конструктора Choropleth и добавляем его на карту
Choropleth(
    geo_data=state_geo,
    data=df_coffee_rat,
    columns=['district', 'rating'],
    key_on='feature.name',
    fill_color='OrRd',
    fill_opacity=0.8,
    legend_name='Рейтинг кофеен по районам',
).add_to(m)

# выводим карту
m

Можем увидеть на первый взгляд что самый высокий рейтинг в центральном и северо-западном округе, а самый худший в западном. Но если подумать то разница практически не заметная максимум 4.3 а минимум 4.2, т.е в целом я бы считал что рейтинги кофеен практически не отличаются.

#### Выясним на какую стоимость чашки капучино стоит ориентироваться при открытии и почему?

In [ ]:
#Выясним среднюю стоимость чашки капучино по районам
df_coffee_avg = df_coffee[['category','district','middle_coffee_cup']]
df_coffee_avg = df_coffee_avg.groupby(['category','district']).agg({'middle_coffee_cup':'mean'}).reset_index()\
.sort_values(by='middle_coffee_cup',ascending=False).round(2)
df_coffee_avg

Построим Столбчатую диаграмму

In [ ]:
#строим столбчатую диаграмму
fig = px.bar(df_coffee_avg, x='district', y= 'middle_coffee_cup', title = 'Стоимость средней чашки капучино по округам',color_discrete_sequence = px.colors.sequential.Sunset)
fig.update_layout(xaxis_title = 'Районы',
                  yaxis_title = 'Кол-во заведений')

fig.show()

Глядя на график можем сделать вывод, что Западный административный округ,Центральный административный округ и Юго-Западный административный округ в лидерах по средней "дороговизне" чашки кофе и разница между ними совсем не существенна,а вот следюущий преследователь - Восточный административный округ уже дешевле от лидера на 15 рублей. Самый дешевый кофе в Юго-Восточном административном округе

#### Предлагаю рассмотреть кол-во посадочных мест по заведениям


In [ ]:
#Кол-во посадочных мест по кофейням
df_coffee_seats = df_coffee.groupby(['category','district']).agg({'seats':'mean'}).reset_index()\
.sort_values(by='seats',ascending=False).round()
df_coffee_seats

In [ ]:
#строим столбчатую диаграмму
fig = px.bar(df_coffee_seats, x='district', y= 'seats', title = 'Среднее кол-во посадочных мест по округам')
fig.update_layout(xaxis_title = 'Категории',
                  yaxis_title = 'Кол-во посадочных мест')

fig.show()

# Рекомендации: 

Исходя из исследования что мы провели выяснилось что в центральном округе слишком высокая конкуренция,по факту кофейни там находятся буквально друг на дружке. Круглосуточных кофеен так же больше всего находится в центральном районе.Что касается рейтинга и цен, то рейтинг в среднем практически идентичен, по ценам опять таки центральный район в топе самых дорогих по средней чашке кофе.

Исходя из этого я бы рекомендовал присмотреться к западной или юго-западной части города. Концепция нашей кофейни - это кофейня по примеру как в сериале "Друзья". Не смотря на то что этот сериал был популярен давно, но практически все молодые люди знают его и понимают что он из себя представляет, соответственно если мы рассмотрим локацию в западной или юго-западной части, там много популярных парков, где гуляют парочки и молодые люди,так же возьмем во внимание МГУ,много иностранных студентов кто так же знаком с особенности этого заведения. Стоит уделить вниманию интерьеру и кол-ву посадочных мест. Мы выяснили что в западной части города в среднем кофейни самые большие по посадке,значит это будет пользоваться спросом,значит есть поток людей. Так же я рекомендовал бы добавить настольные игры и можно проводить какие-то интерактивные мероприятия хотя бы раз в неделю(например мафия или квизы). Это не стандартная кофейня,такие места привлекают в первую очередь атмосферой,ведь вкусный кофе могут готовить много где, но нужно что-то такое, что бы зацепило гостей. Так же можно организовать проектор с показом тех же друзей. 

Что касается цифр, то мы узнали что в западной и юго-западной части города чашка кофе одна из самых дорогих, соответвенно - да, скорее всего и дороже аренда,но точно не дороже чем в центральной части,а вот маржинальность по-моему мнению в этих районах будет одной из самых привлекательных. По рейтингам заведения особо не отличаются, но если сделать все как задумано, то есть большая уверенность в том,что наша кофейня сможет приблизиться к отметке 5!) Так же в этом районе довольно мало круглосуточных кофеен, возможно это и не совсем та специфика для данной категории заведений, но в холодное время года молодым людям(коих там не мало) не охотно проводить время на улице,а в уютной кофейне - это отличный вариант. Возможно стоит поэксперементировать с круглосуточным форматом, но для начала я бы попробовал просто открываться раньше конкурентов(можно добавить завтраки и захватить еще не большую долю рынка) и закрываться позже,что бы люди задерживались по-дольше можно разнообразить меню всякими согревающими напитками, можно устроить те же киносеансы на проекторе, это должно быть что-то "на уютном". Если мы возьмем летнее время года,можно организовать какую-нибудь интересную летнюю веранду,можно поставить бар с выходом на улицу,добавить мягкие мешки,диваны,качели. Можно постойнно декорировать заведение под даты, например хеллуин,новый год,14 февраля и тд.

# Презентация 

https://disk.yandex.ru/d/FtXPNYpkZPQ9hg